In [43]:
from math import *
import numpy as np
import copy
import random

In [57]:
class Tictactoe:
    def __init__(self, board = np.zeros(9)):
        self.acting_player = 1
        self.board = board

    def make_move(self, move):
        if move in range(9) and self.board[move] == 0:
            self.board[move] = self.acting_player
            self.acting_player = 3 - self.acting_player 
            
    def available_moves(self):
        return [i for i in range(9) if (self.board != 'x' and self.board != 'o')]
    
    def check_result(self, player):
        winner = 0
        if player == 1: 
            s = 'x'
        else:
            s = 'o'
        for (a,b,c) in [(0,1,2),(3,4,5),(6,7,8),(0,3,6),(1,4,7),(2,5,8),(0,4,8),(2,4,6)]:
            if self.board[a] == self.board[b] == self.board[c]:
                if self.board[a] == s:
                    return 1.0
                else:
                    return 0.0
        if self.available_moves == []: return 0.5 
        
    def __repr__(self):
        s= ""
        for i in range(9): 
            if self.board[i] == 0:
                s+=str(i+1)
            else:
                s+=str(self.board[i])
            if i == 2 or i == 5: s += "\n"
        return s
        
class MCTSNode: 
    def __init__(self, game_state, parent = None, move = None):
        self.parent = parent
        self.move = move
        self.game_state = game_state
        self.children = []
        self.win_count = 0
        self.num_rollouts = 0
        print('new node')
        print(type(game_state))
        self.unvisited_moves = game_state.available_moves()
        self.player = game_state.acting_player
        
    def add_random_child(self):
        move_index = random.randint(0, len(self.unvisited_moves)-1)
        print('move index', move_index)
        new_move = self.unvisited_moves.pop(move_index)
        print('new mnove', new_move)
        new_game_state = self.game_state.make_move(new_move)
        print('stone', type(new_game_state))
        new_node = MCTSNode(new_game_state, self, new_move)
        print('test')
        print(type(new_node))
        self.children.append(new_node)
        return new_node       
    
    def update(self, result):
        self.win_count += result
        self.num_rollouts += 1
        
class MinimaxAgent:
    def select_move(self, game_state):
        winning_moves = []
        draw_moves = []
        losing_moves = []
        for possible_move in game_state.legal_moves():
            next_state = game_state.apply
    
class UCTAgent:
    def __init__(self, num_rounds = 200, temperature = 1, alphabeta = False):
        self.num_rounds = num_rounds
        self.temperature = temperature
        self.alphabeta = alphabeta
        
    def select_move(self, game_state):
        print ('select move', game_state.acting_player)
        
        print(type(game_state))
        root = MCTSNode(game_state)
        
        for i in range(self.num_rounds):
            node = root
            #gs = copy.copy(game_state) #deepcopy?
            #print('gs')
            #print(type(gs))
            print('node test', node)
            
            
            #selection -- select leaf that has not had playout (simulation) yet
            #while (not node.can_add_child()) and (not node.is_terminal()):
            while node.unvisited_moves == [] and node.children != []:
                print('first')
                node = self.uct_select_child(node)
                print('node test2', node)
                #gs.make_move(node.move)
            
            #expansion -- expand from leaf unless leaf is end of game
            if node.unvisited_moves != []:
                print('second')
                node = node.add_random_child()
                print('node fucked up', node)
                print('fucked', type(node))
            
            #simulation -- complete a random playout from the newly expanded node
            while gs.available_moves() != []:
                print('third')
                gs.make_move(random.choice(gs.available_moves()))
            
            #backpropagation -- update all nodes from the selection to leaf stage
            while node is not None:
                node.update(gs.check_result(node.acting_player))
                node = node.parent
                
    def uct_select_child(self, node):
        total_rollouts = sum(child.num_rollouts for child in node.children)
        log_rollouts = math.log(total_rollouts)
        
        best_score = -1
        best_child = None
        for child in node.children:
            win_percentage = float(child.win_count)/float(child.num_rollouts)
            expl_factor = math.sqrt(log_rollouts / child.num_rollouts)
            uct_score = win_percentage + self.temperature * expl_factor
            if uct_score > best_score:
                best_score = uct_score
                best_child = child
        return best_child
        
class RandomAgent:       
    def select_move(self, game_state):
        return random.choice(game_state.available_moves())
    
class HumanAgent:
    def select_move(self, game_state):
        print('Enter your move (1-9): ')
        input(move)
        if move in game_state.available_moves():
            return move
        else:
            print('Invalid move, try again')
            select_move(game_state)
    
class BasicAgent:
    def select_move(self, game_state):
        return random.choice(game_state.available_moves())
        #win if possible, block loss if possible, do double win option for next time if possible, otherwise random

if __name__ == "__main__":
    game_state = Tictactoe()
    #print(type(game_state))
    #game_state = Tictactoe(board = ['x','o',0,0,'x',0,'o',0,0])
    
    agent1 = UCTAgent(num_rounds = 1000, temperature = 1, alphabeta = False)
    agent2 = UCTAgent(num_rounds = 1000, temperature = 1, alphabeta = False)
    
    #one full game using MCTS for each action
    while (game_state.available_moves() != []):
        print(str(game_state))
        if game_state.acting_player == 1:
            #print(type(game_state))
            ms = agent1.select_move(game_state) #num_rounds tree search iterations from this state
        else:
            ms = agent2.select_move(game_state)
        print("\n" + "Agent " + str(game_state.acting_player) + "move: " + str(ms) + "\n")
        print(game_state)
        game_state.make_move(ms)
    
    if game_state.check_result(game_state.acting_player) == 1.0:
        print("Player " + str(game_state.acting_player) + " wins!")
    elif state.GetResult(game_state.acting_player) == 0.0:
        print("Player " + str(3 - game_state.acting_player) + " wins!")
    else: print("Tie game")
        
        

123
456
789
select move 1
<class '__main__.Tictactoe'>
new node
<class '__main__.Tictactoe'>
gs
<class '__main__.Tictactoe'>
node test <__main__.MCTSNode object at 0x1126bdda0>
second
move index 1
new mnove 1
stone <class 'NoneType'>
new node
<class 'NoneType'>


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if sys.path[0] == '':


AttributeError: 'NoneType' object has no attribute 'available_moves'

In [ ]:
def show_tree(node, indent='', max_depth=3):
    if max_depth < 0:
        return
    if node is None:
        return
    if node.parent is None:
        print('%sroot' % indent)
    else:
        player = node.parent.game_state.next_player
        move = node.move
        print('%s%s %s %d %.3f' % (
            indent, fmt(player), fmt(move),
            node.num_rollouts,
            node.winning_frac(player),
        ))
    for child in sorted(node.children, key=lambda n: n.num_rollouts, reverse=True):
        show_tree(child, indent + '  ', max_depth - 1)



In [21]:
game_state.type()

AttributeError: 'Tictactoe' object has no attribute 'type'

In [3]:
for i in range(8):
    print(i)

0
1
2
3
4
5
6
7


In [6]:
i = np.zeros(9)

In [7]:
print(i)

[0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [9]:
8 in range(9)

True

In [11]:
a = [1, 2, 3]

In [12]:
len(a)

3